### 지도학습

#### 분류 - 이진분류

In [1]:
import pandas as pd

In [2]:
df_TFD = pd.read_csv('../../../datasets/TitanicFromDisaster_train.csv')
df_TFD[:2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


#### 전처리

In [3]:
df_TFD.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
df_TFD_extract = df_TFD[['Survived', 'Pclass', 'Age']]
df_TFD_extract.isnull().sum()

Survived      0
Pclass        0
Age         177
dtype: int64

In [5]:
df_TFD_extract_preprocess = df_TFD_extract.dropna()
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age
0,0,3,22.0
1,1,1,38.0


#### Scaling & Encoding

In [6]:
df_TFD_extract_preprocess['Pclass'].value_counts()

3    355
1    186
2    173
Name: Pclass, dtype: int64

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
oneHotEncoder = OneHotEncoder()
oneHotEncoder.fit(df_TFD_extract_preprocess[['Pclass']])  #해당 항목 학습한 것임

OneHotEncoder()

In [9]:
columns_name = oneHotEncoder.categories_

In [10]:
# oneHotEncoder.transform(df_TFD_extract_preprocess[['Pclass']]).toarray()  #실제 값 확인용
encoded_data = oneHotEncoder.transform(df_TFD_extract_preprocess[['Pclass']]).toarray()

In [11]:
encoded_data.shape

(714, 3)

In [22]:
df_encoded_data = pd.DataFrame(data=encoded_data,columns=columns_name)
df_encoded_data[:2]

,1,2,3
0,0.0,0.0,1.0
1,1.0,0.0,0.0


In [25]:
df_encoded_data.index , df_encoded_data.shape

(RangeIndex(start=0, stop=714, step=1), (714, 3))

In [27]:
df_TFD_extract_preprocess.index , df_TFD_extract_preprocess.shape

(Int64Index([  0,   1,   2,   3,   4,   6,   7,   8,   9,  10,
             ...
             653, 656, 667, 669, 674, 680, 692, 697, 709, 711],
            dtype='int64', length=861),
 (861, 6))

In [14]:
#df_encoded_data = pd.get_dummies(df_TFD_extract_preprocess['Pclass'],prefix='Pclass')  #임시로 get_dummies 사용

In [15]:
df_TFD_extract_preprocess = pd.concat([df_TFD_extract_preprocess,df_encoded_data],axis=1)
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age,"(1,)","(2,)","(3,)"
0,0.0,3.0,22.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,0.0


In [16]:
df_TFD_extract_preprocess[['Pclass']][:3]

,Pclass
0,3.0
1,1.0
2,3.0


In [17]:
df_TFD_extract_preprocess.columns

Index(['Survived', 'Pclass', 'Age', (1,), (2,), (3,)], dtype='object')

In [18]:
target = df_TFD_extract_preprocess['Survived']

In [19]:
features = df_TFD_extract_preprocess.drop(columns=['Survived','Pclass'])
features.columns

Index(['Age', (1,), (2,), (3,)], dtype='object')

#### MinMaxScaler

In [20]:
from sklearn.preprocessing import MinMaxScaler

In [21]:
minMaxScaler = MinMaxScaler() # 인스턴스화
minMaxScaler.fit_transform(features)

TypeError: Feature names are only supported if all input features have string names, but your input has ['str', 'tuple'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

#### 정형화 단계

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111)
features_train.shape, target_train.shape, features_test.shape, target_test.shape

#### 모델학습

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(features_train, target_train)

In [ ]:
model.coef_, model.intercept_
# (array([[-0.04517542,  0.62299973,  0.00845763, -0.63151587,  0.62299973, 0.00845763, -0.63151587]]),
# array([1.27195902])) 정규화 이전값

#### 예측

In [ ]:
df_TFD_extract_preprocess[10:15] # index가 features_train 같다.

In [ ]:
model.predict(features_train[10:15])

In [ ]:
model.predict_proba(features_train[10:15])

#### 평가

In [ ]:
target_train_predict = model.predict(features_train)
target_train_predict.shape # target_train.shape 동일

In [ ]:
from sklearn.metrics import accuracy_score # 정확도

In [ ]:
accuracy_score(target_train, target_train_predict) # 교내 시험
# 0.7065420560747664 정규화 이전

In [ ]:
target_test_perdict = model.predict(features_test)
target_test_perdict.shape # target_test.shape 동일

In [ ]:
accuracy_score(target_test, target_test_perdict) # 교외 시험
# 0.659217877094972 정규화 이전

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(target_train, target_train_predict))

In [ ]:
print(classification_report(target_test, target_test_perdict))

#### 오차 행렬

In [ ]:
from sklearn.metrics import confusion_matrix ,precision_score , recall_score , f1_score

In [ ]:
confusion_matrix(target_train, target_train_predict)

In [ ]:
precision_score(target_train,target_train_predict)

In [ ]:
recall_score(target_train,target_train_predict)